In [6]:
import pandas as pd
import openpyxl # type: ignore
import unicodedata

Funciones

In [4]:
# Función para limpiar valores
def limpiar_valores(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', errors='ignore').decode('utf-8')
        value = value.strip()
    elif pd.isnull(value):
        # Sustituir null por un espacio en blanco
        value = ''
    return value

# Función para clasificar en quinquenios
def clasificar(edad):
    if pd.isna(edad):
        return "desconocido"
    inicio = (edad // 5) * 5
    fin = inicio + 4
    return f'{inicio}-{fin}'

In [7]:
# Cargar el archivo Excel y limpiar valores
df = pd.read_excel("E://files_pandas//dengue//DENGUE141024.xlsx")
df = df.applymap(limpiar_valores)

# Filtrar valores específicos
df = df[df["DES_INS_UNIDAD"].str.strip().isin(["IMSS_OPD", "SSA"])]


C:\Users\najer\AppData\Local\Temp\ipykernel_16220\859555970.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_valores)


In [ ]:
import pandas as pd
import folium
import re

# Función para convertir de DMS a decimal
def dms_to_decimal(dms):
    # Usar regex para extraer los componentes de grados, minutos, y segundos
    parts = re.split('[°\'"]+', dms)
    if len(parts) == 4:
        degrees, minutes, seconds, direction = parts
    else:
        degrees, minutes, seconds = parts
        direction = dms[-1]
    
    decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

# Cargar el archivo CSV
data = pd.read_csv("E://files_pandas//sisver//AGEEML_202410311056515.csv")

# Filtrar y seleccionar columnas necesarias
data = data[data['NOM_ENT'] == "Hidalgo"]
data = data[['NOM_LOC', 'LATITUD', 'LONGITUD']]
data = data.dropna(subset=['NOM_LOC', 'LATITUD', 'LONGITUD'])

# Convertir las coordenadas
data['LATITUD'] = data['LATITUD'].apply(dms_to_decimal)
data['LONGITUD'] = data['LONGITUD'].apply(dms_to_decimal)

# Crear el mapa centrado en la ubicación media de los datos
mapa = folium.Map(location=[data['LATITUD'].mean(), data['LONGITUD'].mean()], zoom_start=12)

# Añadir puntos al mapa
for _, row in data.iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=row['NOM_LOC']
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save('mapa_localidades.html')

# Mostrar mensaje de éxito
print("El mapa se ha guardado exitosamente en 'mapa_localidades.html'")
